In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sea
import statistics
from scipy.stats import sem, t, ttest_ind_from_stats, probplot
from scipy import mean,stats
import numpy as np
import datetime
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
import yfinance as yf
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

stock=["DIS","ROK","BUD","XOM","CVS"]
start = datetime.datetime(2018,11,1)
end = datetime.datetime(2019,10,31)
df = web.get_data_yahoo(stock, start=start , end=end)[["Open","Close"]]
df.head()

b = widgets.Dropdown(options = ["May 2019 - Oct 2019",'Nov 2018 - Apr 2019', 'Nov 2018 - Oct 2019'], value = 'Nov 2018 - Oct 2019', description = "Time period")
a = widgets.Dropdown(options=["--Stocks--","DIS","ROK","BUD","XOM",'CVS'], value = '--Stocks--', description = "Select Stock")
d = widgets.Dropdown(options=['--Stock2--',"DIS","ROK","BUD","XOM",'CVS'], value = '--Stock2--',description = "SelectStock2")
c = widgets.Dropdown(options=['--Function--','Histogram of Log returns','Normal Probablity Plot','Confidence interval for means and variances', 'Regression of the log returns on time', 'Hypothesis test for population means','Regression of log return of one stock on another','Old Wives Tale Test'], value = '--Function--' ,description = 'Functions')
e = widgets.RadioButtons(options=['0.1', '0.05'], value= '0.05',description='α:',disabled= False)
df['date1'] = [i for i in range(1,len(df)+1)]
def f(a, b, c, d, e):
    
    if a != '--Stocks--' and c == 'Histogram of Log returns' and d == "--Stock2--":  
        df['log_returns']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        if b == "May 2019 - Oct 2019":
            plt.hist(df.loc['2019-05-01': '2019-10-31', 'log_returns'], bins = 'fd')        
        elif b == 'Nov 2018 - Apr 2019':
            plt.hist(df.loc['2018-11-01': '2019-04-30', 'log_returns'], bins = 'fd') 
        elif b == 'Nov 2018 - Oct 2019':
            plt.hist(df.loc['2018-11-01': '2019-10-31', 'log_returns'], bins = 'fd')
        plt.xlabel("Daily Returns")
        plt.ylabel("Price")
        print(f"Log returns for {a} for the time period {b}")
    elif a != '--Stocks--' and c == 'Histogram of Log returns' and d != "--Stock2--":
        print("Histogram can only be displayed for a single stock entry, please select \n --Stock2-- in SelectStock2 dropdown")
    
    
    elif a != '--Stocks--' and c == "Normal Probablity Plot" and d == "--Stock2--":
        df['log_returns']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        if b == "May 2019 - Oct 2019":
            p = probplot(df.loc['2019-05-01': '2019-10-31', 'log_returns'], dist="norm", plot = plt)
        elif b == 'Nov 2018 - Apr 2019':
            p = probplot(df.loc['2018-11-01': '2019-04-30', 'log_returns'], dist="norm", plot = plt) 
        elif b == 'Nov 2018 - Oct 2019':
            p = probplot(df.loc['2018-11-01': '2019-10-31', 'log_returns'], dist="norm", plot = plt)
        plt.show(p)
        print(f"Normal Probablity Plot for {a} for the time period {b}")
        print(f"\nSince the graph obtained is a straight line, the data of stock {a} is \nnormally distributed")
    elif a != '--Stocks--' and c == "Normal Probablity Plot" and d != "--Stock2--":
        print("Normal Probablity Plots can only be displayed for a single stock entry,\n please select --Stock2-- in SelectStock2 dropdown")
    
    
    elif a != '--Stocks--' and c == 'Confidence interval for means and variances':
        df['log_returns3']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        Bucket1 = df.loc['2018-11-01': '2019-04-30', 'log_returns3']
        Bucket2 = df.loc['2019-05-01': '2019-10-31', 'log_returns3']
        Bucket3 = df.loc['2018-11-01': '2019-10-31', 'log_returns3']
        if b == 'Nov 2018 - Apr 2019':
            data = Bucket1
        elif b == "May 2019 - Oct 2019":
            data = Bucket2
        elif b == 'Nov 2018 - Oct 2019':
            data = Bucket3
        confidence = ( 1 - float(e)) 
        n = len(data)
        m = mean(data)
        std_err = sem(data)
        h = std_err * t.ppf((1 + confidence) / 2, n - 1)
        start = m - h
        start
        end = m + h
        end
        alpha = float(e)               
        n = len(data)               
        s2 = np.var(data, ddof=1)  
        df1 = n - 1                 
        upper = (n - 1) * s2 / stats.chi2.ppf(alpha / 2, df1)
        lower = (n - 1) * s2 / stats.chi2.ppf(1 - alpha / 2, df1)
        print(f"The confidence interval for mean of {a} with α = {e} is: \n ({start},{end})\n")
        print(f"The confidence interval for variance of {a} with α = {e} is: \n ({lower}, {upper})")
    
    
    elif a != '--Stocks--' and d!= '--Stock2--' and c == 'Hypothesis test for population means' and d != a and e=='0.05':
        df['log_returnsA']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        df['log_returnsB']  = np.log(df['Close'][d]) - np.log(df['Open'][d])
        e1 = df['log_returnsA']
        f = df['log_returnsB']
        std1 = statistics.stdev(e1)
        std2 = statistics.stdev(f)
        t1 = ttest_ind_from_stats(mean(e1), std1 = std1, nobs1=len(e1) ,mean2=mean(f), std2=std2, nobs2=len(f))
        print(f"Ho: μ{a} = μ{d} vs H1: μ{a} = μ{d}")
        print("Test Statistics:",t1[0])
        print("P-value :" ,t1[1])
        if t1[1]>= float(e) and abs(t1[0]) <= 1.9679:
            print(f"We cannot reject the hypothesis as |TS| <= 1.9679 and P-value >= {float(e)}")
        else:
            print("We will have to reject the hypothesis")
    elif a != '--Stocks--' and d == '--Stock2--' and c == 'Hypothesis test for population means':
        print("Hypothesis test requires selection of two stocks\nPlease Select a stock from SelectStock2")
    elif a != '--Stocks--' and d == a and c == 'Hypothesis test for population means':
        print("Cannot perform operation on same stock\nPlease select a different stock from SelectStock2")
    elif a != '--Stocks--' and d!= '--Stock2--' and c == 'Hypothesis test for population means' and d != a and e=='0.1':
        print("Please select standard α value of 0.05")
    
    
    elif a != "--Stocks--" and c == 'Regression of log return of one stock on another' and d != "--Stock2--" and a!=d:
        df['log_returnsA']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        df['log_returnsB']  = np.log(df['Close'][d]) - np.log(df['Open'][d])
        if b == "May 2019 - Oct 2019":
            X=df.loc['2019-05-01': '2019-10-31', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2019-05-01': '2019-10-31', 'log_returnsB'].values.reshape(-1,1)
        elif b == 'Nov 2018 - Apr 2019':
            X=df.loc['2018-11-01': '2019-04-30', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2018-11-01': '2019-04-30', 'log_returnsB'].values.reshape(-1,1) 
        elif b == 'Nov 2018 - Oct 2019':
            X=df.loc['2018-11-01': '2019-10-31', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2018-11-01': '2019-10-31', 'log_returnsB'].values.reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        regressor = LinearRegression()  
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        dfpredict = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
        plt.scatter(X_test, y_test,  color='gray')
        plt.plot(X_test, y_pred, color='red', linewidth=2)
        plt.show()
        print("Regressor Intercept:",regressor.intercept_)
        print("Regressor Coefficient:", regressor.coef_)
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    elif a != "--Stocks--" and c == 'Regression of log return of one stock on another' and d == "--Stock2--":
        print("Please select a stock from SelectStock2 to perform Regression")
    elif a != "--Stocks--" and c == 'Regression of log return of one stock on another' and d == a:
        print("Please select two different stocks to perform Regression")
    
    
    elif a != "--Stocks--" and c == 'Regression of the log returns on time' and d == '--Stock2--' :
        df['log_returnsA']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        if b == "May 2019 - Oct 2019":
            X=df.loc['2019-05-01': '2019-10-31', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2019-05-01': '2019-10-31', 'date1'].values.reshape(-1,1)
        elif b == 'Nov 2018 - Apr 2019':
            X=df.loc['2018-11-01': '2019-04-30', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2018-11-01': '2019-04-30', 'date1'].values.reshape(-1,1)
        elif b == 'Nov 2018 - Oct 2019':
            X=df.loc['2018-11-01': '2019-10-31', 'log_returnsA'].values.reshape(-1,1)
            y=df.loc['2018-11-01': '2019-10-31', 'date1'].values.reshape(-1,1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
        regressor = LinearRegression()  
        regressor.fit(X_train, y_train)
        y_pred = regressor.predict(X_test)
        dfpredict = pd.DataFrame({'Actual': y_test.flatten(), 'Predicted': y_pred.flatten()})
        plt.scatter(X_test, y_test,  color='gray')
        plt.plot(X_test, y_pred, color='red', linewidth=2)
        plt.show()
        print("Regressor Intercept:",regressor.intercept_)
        print("Regressor Coefficient:", regressor.coef_)
        print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
        print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
        print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
    elif a != "--Stocks--" and c == 'Regression of the log returns on time' and d != '--Stock2--':
        print("To perform Time Series Regression, only one stock value is considered\nplease select --Stock2-- in SelectStock2 dropdown")
    
    
    elif a != "--Stocks--" and c == 'Old Wives Tale Test' and d == '--Stock2--' and (b!="May 2019 - Oct 2019" or b!= 'Nov 2018 - Apr 2019') and b=='Nov 2018 - Oct 2019':
        df['log_returns4']  = np.log(df['Close'][a]) - np.log(df['Open'][a])
        Bucket1 = df.loc['2018-11-01': '2019-04-30', 'log_returns4']
        Bucket2 = df.loc['2019-05-01': '2019-10-31', 'log_returns4'] 
        std1 = statistics.stdev(Bucket1)
        std2 = statistics.stdev(Bucket2)
        t2 =ttest_ind_from_stats(mean(Bucket1), std1 = std1, nobs1=len(Bucket1) ,mean2=mean(Bucket2), std2=std2, nobs2=len(Bucket2))
        
        print("Test Statistics:",t2[0])
        print("P-value :" ,t2[1])
        print(f"Ho: μ(Nov 2018 - Apr 2019) =< μ(May 2019 - Oct 2019) \n H1: μ(Nov 2018 - Apr 2019) > μ(May 2019 - Oct 2019)")
        if t2[1] >= float(e) and abs(t2[0]) < 1.651:
            print(f"We cannot reject the hypothesis as |TS| <= 1.651 and P-value >= {float(e)}")
        else:
            print("We will have to reject the hypothesis")
    elif a != "--Stocks--" and c == 'Old Wives Tale Test' and d != '--Stock2--':
        print("Hypothesis test is only carried using single stock /nPlease select --Stock2-- option from SelectStock2")
    elif a != "--Stocks--" and c == 'Old Wives Tale Test' and (b=="May 2019 - Oct 2019" or b== 'Nov 2018 - Apr 2019') and (d != '--Stock2--' or d == '--Stock2--'):
        print(f"Hypothesis test can only be carried out on the entire time frame \nPlease select time 'Nov 2018 - Oct 2019' from time period")
        
        
out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c, 'd': d, 'e': e})


widgets.HBox([widgets.VBox([a, b, c, d, e]), out])